In [ ]:
#https://www.kaggle.com/c/customer-churn-prediction-2020


#Ce notebook atteint le Top 13 sans hyperparameting  https://www.kaggle.com/c/customer-churn-prediction-2020/leaderboard

In [1]:
import time
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.cluster import KMeans
from kneed import KneeLocator

In [2]:
pd.set_option('max_rows', 120)
pd.set_option('max_colwidth', 120)

In [65]:
df_churn = pd.read_csv('train.csv')
df_churn = df_churn.rename(columns=str.lower)
df_churn.head().T

,0,1,2,3,4
state,OH,NJ,OH,OK,MA
account_length,107,137,84,75,121
area_code,area_code_415,area_code_415,area_code_408,area_code_415,area_code_510
international_plan,no,no,yes,yes,no
voice_mail_plan,yes,no,no,no,yes
number_vmail_messages,26,0,0,0,24
total_day_minutes,161.60,243.40,299.40,166.70,218.20
total_day_calls,123,114,71,113,88
total_day_charge,27.47,41.38,50.90,28.34,37.09
total_eve_minutes,195.50,121.20,61.90,148.30,348.50


In [4]:
df_churn.isna().sum()

state                            0
account_length                   0
area_code                        0
international_plan               0
voice_mail_plan                  0
number_vmail_messages            0
total_day_minutes                0
total_day_calls                  0
total_day_charge                 0
total_eve_minutes                0
total_eve_calls                  0
total_eve_charge                 0
total_night_minutes              0
total_night_calls                0
total_night_charge               0
total_intl_minutes               0
total_intl_calls                 0
total_intl_charge                0
number_customer_service_calls    0
churn                            0
dtype: int64

In [5]:
df_churn.dropna()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4245,MT,83,area_code_415,no,no,0,188.3,70,32.01,243.8,88,20.72,213.7,79,9.62,10.3,6,2.78,0,no
4246,WV,73,area_code_408,no,no,0,177.9,89,30.24,131.2,82,11.15,186.2,89,8.38,11.5,6,3.11,3,no
4247,NC,75,area_code_408,no,no,0,170.7,101,29.02,193.1,126,16.41,129.1,104,5.81,6.9,7,1.86,1,no
4248,HI,50,area_code_408,no,yes,40,235.7,127,40.07,223.0,126,18.96,297.5,116,13.39,9.9,5,2.67,2,no


In [6]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4250 non-null   object 
 1   account_length                 4250 non-null   int64  
 2   area_code                      4250 non-null   object 
 3   international_plan             4250 non-null   object 
 4   voice_mail_plan                4250 non-null   object 
 5   number_vmail_messages          4250 non-null   int64  
 6   total_day_minutes              4250 non-null   float64
 7   total_day_calls                4250 non-null   int64  
 8   total_day_charge               4250 non-null   float64
 9   total_eve_minutes              4250 non-null   float64
 10  total_eve_calls                4250 non-null   int64  
 11  total_eve_charge               4250 non-null   float64
 12  total_night_minutes            4250 non-null   f

In [ ]:
#df_churn['account_length']=np.log1p(df_churn['account_length'])
#df_churn['number_vmail_messages']=np.log1p(df_churn['number_vmail_messages'])
#df_churn['total_day_minutes']=np.log1p(df_churn['total_day_minutes'])
#df_churn['total_day_calls']=np.log1p(df_churn['total_day_calls'])
#df_churn['total_day_charge']=np.log1p(df_churn['total_day_charge'])
#df_churn['total_eve_minutes']=np.log1p(df_churn['total_eve_minutes'])
#df_churn['total_eve_calls']=np.log1p(df_churn['total_eve_calls'])
#df_churn['total_eve_charge']=np.log1p(df_churn['total_eve_charge'])
#df_churn['total_night_minutes']=np.log1p(df_churn['total_night_minutes'])
#df_churn['total_night_calls']=np.log1p(df_churn['total_night_calls'])
#df_churn['total_night_charge']=np.log1p(df_churn['total_night_charge'])
#df_churn['total_intl_minutes']=np.log1p(df_churn['total_intl_minutes'])
#df_churn['total_intl_calls']=np.log1p(df_churn['total_intl_calls'])
#df_churn['total_intl_charge']=np.log1p(df_churn['total_intl_charge'])
#df_churn['number_customer_service_calls']=np.log1p(df_churn['number_customer_service_calls'])

In [ ]:
#kmeans = KMeans(n_clusters=3)
#kmeans.fit(df_churn[['total_day_minutes','total_day_calls','total_day_charge']])


In [ ]:
#df_churn['total_day_all'] = kmeans.predict(df_churn[['total_day_minutes','total_day_calls','total_day_charge']])

In [ ]:
#kmeans1 = KMeans(n_clusters=3)
#kmeans1.fit(df_churn[['total_eve_minutes','total_eve_calls','total_eve_charge']])

In [ ]:
#df_churn['total_eves']=kmeans1.predict(df_churn[['total_eve_minutes','total_eve_calls','total_eve_charge']])

In [7]:
df_churn['churn'] = df_churn['churn'].replace(('yes', 'no'), (1, 0))

In [8]:
df_churn['churn'].value_counts()

0    3652
1     598
Name: churn, dtype: int64

In [9]:
df_churn['international_plan'] = df_churn['international_plan'].replace(('yes', 'no'), (1, 0))
df_churn['voice_mail_plan'] = df_churn['voice_mail_plan'].replace(('yes', 'no'), (1, 0))

In [10]:
def calculate_call_charge_rate(df, colA, colB):
    return df[colB] / df[colA]

In [11]:
df_churn['charge_rate_day'] = calculate_call_charge_rate(df_churn, 'total_day_minutes', 'total_day_charge')
df_churn['charge_rate_night'] = calculate_call_charge_rate(df_churn, 'total_night_minutes', 'total_night_charge')
df_churn['charge_rate_intl'] = calculate_call_charge_rate(df_churn, 'total_intl_minutes', 'total_intl_charge')
df_churn['charge_rate_eve'] = calculate_call_charge_rate(df_churn, 'total_eve_minutes', 'total_eve_charge')

In [12]:
def _mean_encoding(df, colA, target):
    mean_encodage= df.groupby(colA)[target].mean()
    return df[colA].map(mean_encodage)

In [13]:
df_churn['mean_state_encoded'] = _mean_encoding(df_churn, 'state', 'churn')
df_churn['mean_international_plan_encoded'] =_mean_encoding(df_churn, 'international_plan', 'churn')
df_churn['mean_voice_mail_plan_encoded'] = _mean_encoding(df_churn, 'voice_mail_plan', 'churn')

In [14]:
df_churn = pd.get_dummies(df_churn, columns=['state','area_code'])

In [15]:
df_churn

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,...,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510
0,107,0,1,26,161.6,123,27.47,195.5,103,16.62,...,0,0,0,0,0,0,0,0,1,0
1,137,0,0,0,243.4,114,41.38,121.2,110,10.30,...,0,0,0,0,0,0,0,0,1,0
2,84,1,0,0,299.4,71,50.90,61.9,88,5.26,...,0,0,0,0,0,0,0,1,0,0
3,75,1,0,0,166.7,113,28.34,148.3,122,12.61,...,0,0,0,0,0,0,0,0,1,0
4,121,0,1,24,218.2,88,37.09,348.5,108,29.62,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4245,83,0,0,0,188.3,70,32.01,243.8,88,20.72,...,0,0,0,0,0,0,0,0,1,0
4246,73,0,0,0,177.9,89,30.24,131.2,82,11.15,...,0,0,0,0,0,1,0,1,0,0
4247,75,0,0,0,170.7,101,29.02,193.1,126,16.41,...,0,0,0,0,0,0,0,1,0,0
4248,50,0,1,40,235.7,127,40.07,223.0,126,18.96,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#df_churn[df_churn.columns[1:]].corr()['churn'][:].sort_values(ascending=False).to_frame()

In [ ]:
#df_churn = df_churn.reset_index()'international_plan'mean_international_plan_encoded

In [17]:
X = df_churn.drop(['churn','international_plan' ], axis=1)
y = df_churn['churn']


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0,
                                                    stratify=y
                                                   )

In [19]:
def go_inside_pipeline(X, model):
    """Retourne un pipeline pour prétraitement des données et les regrouper avec un modèle.
    
    Arguments:
        X : donnee apprentissaqge 
        model (object): scikit-learn model object
    
    Retour: 
    les  etapes du pipeline. 
    """
    
    num_columns = list(X.select_dtypes(exclude=['object']).columns.values.tolist())    
    categ_columns = list(X.select_dtypes(include=['object']).columns.values.tolist())
    num_pipeline = SimpleImputer(strategy='constant')
    categ_pipeline = OneHotEncoder(handle_unknown='ignore')
    #MinMaxScaler()
    

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', num_pipeline, num_columns),
            ('categorical', categ_pipeline, categ_columns),
            ], remainder='passthrough'
    )
# RFE(estimator=DecisionTreeClassifier(), n_features_to_select=9)RFECV(estimator=XGBClassifier())RandomForestClassifier()
#SMOTE(random_state=1, k_neighbors=4)BorderlineSMOTE(random_state=0)MinMaxScalerStandardScaler
    steps_pipeline = imbpipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote',SMOTE(random_state=1) ),
        ('scaler',MinMaxScaler()),
        ('feature_selection', RFE(estimator=DecisionTreeClassifier(), n_features_to_select=9)),
        
        ('model', model)
    ])
    
    return steps_pipeline

In [20]:
model = XGBClassifier()
test_pipeline = go_inside_pipeline(X_train, model)
test_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  SimpleImputer(strategy='constant'),
                                                  ['account_length',
                                                   'voice_mail_plan',
                                                   'number_vmail_messages',
                                                   'total_day_minutes',
                                                   'total_day_calls',
                                                   'total_day_charge',
                                                   'total_eve_minutes',
                                                   'total_eve_calls',
                                                   'total_eve_charge',
                                                   'total_night_minutes',
                                             

In [ ]:
scale_pos_weight = round((len(df_churn['churn']) - sum(df_churn['churn'])) / sum(df_churn['churn']))
scale_pos_weight

In [21]:
def selectionner_le_meilleur_model(X, y, pipeline=None):
    """Teste un ensemble de  classifieurs and retourne leur performance sur la donnee d'apprentissage.
    
    Arguments:
        X (type object): un dataframe contenant donnee apprentissage. 
        y (type object): un dataframe contenant label d'apprentissage. 
        pipeline (object): Pipeline sortant de go_inside_pipeline().

    Sortie:
        df (object):un dataframe contenant les performances du model et les resul. 
    """
    all_classifiers = {}
    all_classifiers.update({"DummyClassifier": DummyClassifier(strategy='most_frequent')})
    all_classifiers.update({"KNeighborsClassifier": KNeighborsClassifier()})
    all_classifiers.update({"RidgeClassifier": RidgeClassifier()})
    all_classifiers.update({"SGDClassifier": SGDClassifier()})
    all_classifiers.update({"BaggingClassifier": BaggingClassifier()})
    all_classifiers.update({"BernoulliNB": BernoulliNB()})
    all_classifiers.update({"SVC": SVC()})
    all_classifiers.update({"CatBoostClassifier":CatBoostClassifier(silent=True)})
    all_classifiers.update({"XGBClassifier": XGBClassifier(use_label_encoder=False, 
                                                       eval_metric='logloss',
                                                       objective='binary:logistic',
                                                      )})
    all_classifiers.update({"LGBMClassifier": LGBMClassifier()})
    all_classifiers.update({"RandomForestClassifier": RandomForestClassifier()})
    all_classifiers.update({"DecisionTreeClassifier": DecisionTreeClassifier()})
    all_classifiers.update({"ExtraTreeClassifier": ExtraTreeClassifier()})
    all_classifiers.update({"ExtraTreesClassifier": ExtraTreeClassifier()})    
    all_classifiers.update({"AdaBoostClassifier": AdaBoostClassifier()})
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
    models.append(('BaggingClassifier', BaggingClassifier()))
    all_classifiers.update({"VotingClassifier (XGBClassifier, CatBoostClassifier, BaggingClassifier)": VotingClassifier(models)})
    
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
    all_classifiers.update({"VotingClassifier (XGBClassifier, CatBoostClassifier)": VotingClassifier(models)})
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('LGBMClassifier', LGBMClassifier()))
    all_classifiers.update({"VotingClassifier (XGBClassifier, LGBMClassifier)": VotingClassifier(models)})
    
    models = []
    models.append(('LGBMClassifier', LGBMClassifier()))
    models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
    all_classifiers.update({"VotingClassifier (LGBMClassifier, CatBoostClassifier)": VotingClassifier(models)})
    models=[]
    #voting1=VotingClassifier(estimators=[('LGBMClassifier', LGBMClassifier()), 
    #                                     ('CatBoostClassifier', CatBoostClassifier(silent=True))])
    #voting2=VotingClassifier(estimators=[('XGBClassifier', XGBClassifier(use_label_encoder=False,eval_metric='logloss',
    #                                                                     objective='binary:logistic'),('LGBMClassifier',LGBMClassifier()))
    #                                     ])
    #voting3=VotingClassifier(estimators=[
        #('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss',objective='binary:logistic')),
        #('LGBMClassifier', LGBMClassifier()),
        #('CatBoostClassifier', CatBoostClassifier(silent=True))])
    #models.append(('voting1',voting1))
    #models.append(('voting2',voting2))
    #models.append(('voting3',voting3))
    #all_classifiers.update({"ensemblemodel": VotingClassifier(models)})
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('LGBMClassifier', LGBMClassifier()))
    models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
    all_classifiers.update({"VotingClassifier (XGBClassifier, LGBMClassifier, CatBoostClassifier)": VotingClassifier(models)})
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('RandomForestClassifier', RandomForestClassifier()))
    models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
    all_classifiers.update({"VotingClassifier (XGBClassifier, RandomForestClassifier, DecisionTreeClassifier)": VotingClassifier(models)})
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('AdaBoostClassifier', AdaBoostClassifier()))
    models.append(('ExtraTreeClassifier', ExtraTreeClassifier()))
    all_classifiers.update({"VotingClassifier (XGBClassifier, AdaBoostClassifier, ExtraTreeClassifier)": VotingClassifier(models)})
    
    
    models = []
    models.append(('XGBClassifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
    models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
    all_classifiers.update({"VotingClassifier (XGBClassifier, ExtraTreesClassifier)": VotingClassifier(models)})    
    
    dataframe_models = pd.DataFrame(columns=['Model', 'Time', 'Accuracy'])
    for cle in all_classifiers:
        debut = time.time()

        pipeline = go_inside_pipeline(X_train, all_classifiers[cle])
        
        cv = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
        

        row = {'Model': cle,
               'Time': format(round((time.time() -debut)/60,2)),
               'Accuracy': cv.mean(),
        }

        dataframe_models  = dataframe_models.append(row, ignore_index=True)
        
    dataframe_models  = dataframe_models.sort_values(by='Accuracy', ascending=False)
    return dataframe_models
    
    
                             
                             
                             
    
    
    
    

In [22]:
models = selectionner_le_meilleur_model(X_train, y_train)
models.head(23)

,Model,Time,Accuracy
17,"VotingClassifier (XGBClassifier, LGBMClassifier)",0.98,0.962059
16,"VotingClassifier (XGBClassifier, CatBoostClassifier)",1.88,0.961765
19,"VotingClassifier (XGBClassifier, LGBMClassifier, CatBoostClassifier)",2.11,0.960000
20,"VotingClassifier (XGBClassifier, RandomForestClassifier, DecisionTreeClassifier)",1.17,0.959706
7,CatBoostClassifier,2.4,0.959412
15,"VotingClassifier (XGBClassifier, CatBoostClassifier, BaggingClassifier)",1.84,0.959118
8,XGBClassifier,1.15,0.958824
18,"VotingClassifier (LGBMClassifier, CatBoostClassifier)",1.94,0.957647
22,"VotingClassifier (XGBClassifier, ExtraTreesClassifier)",1.11,0.954412
10,RandomForestClassifier,1.16,0.954412


In [ ]:
stacked_models = []
#stacked_models.append(('XGBClassifier',XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
stacked_models.append(('LGBMClassifier', LGBMClassifier()))
stacked_models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
stacked_model = VotingClassifier(stacked_models)
sortie_pipeline=features_names = go_inside_pipeline(X_train, stacked_model)
sortie_pipeline.fit(X_train, y_train)


In [23]:
#############################Deuxiemme############
import pickle


stacked_models = []
#stacked_models.append(('XGBClassifier',XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
stacked_models.append(('LGBMClassifier', LGBMClassifier()))
stacked_models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
#stacked_models.append(('BaggingClassifier', BaggingClassifier()))
stacked_model = VotingClassifier(stacked_models)
sortie_pipeline=features_names = go_inside_pipeline(X_train, stacked_model)
sortie_pipeline.fit(X_train, y_train)
with open('model1.pkl', 'wb') as f:
    pickle.dump(sortie_pipeline, f)
y_pred = sortie_pipeline.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
#f1_score = f1_score(y_test, y_pred)
print('ROC/AUC:', roc_auc)
print('Accuracy:', accuracy)
print('*****************************')
print(classification_report(y_test, y_pred))

ROC/AUC: 0.8965753424657534
Accuracy: 0.9658823529411765
*****************************
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       730
           1       0.95      0.80      0.87       120

    accuracy                           0.97       850
   macro avg       0.96      0.90      0.92       850
weighted avg       0.97      0.97      0.96       850



In [ ]:
features_names.named_steps['feature_selection'].get_support()

In [24]:
best_features=list(X.columns[features_names.named_steps['feature_selection'].get_support()])
best_features




['number_vmail_messages',
 'total_day_minutes',
 'total_day_charge',
 'total_eve_charge',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'number_customer_service_calls',
 'mean_international_plan_encoded']

In [27]:
#For apps
df_app=df_churn[best_features]
#df_app['churn'] = df_app['churn'].replace(('yes', 'no'), (1, 0))
#df_app['international_plan'] = df_app['international_plan'].replace(('yes', 'no'), (1, 0))
y = df_churn['churn']
X_train, X_test, y_train, y_test = train_test_split(df_app, y, 
                                                    test_size=0.2, 
                                                    random_state=0,
                                                    stratify=y
                                                   )
def go_inside_pipeline_for_app(X, model):
    """Retourne un pipeline pour prétraitement des données et les regrouper avec un modèle.
    
    Arguments:
        X : donnee apprentissaqge 
        model (object): scikit-learn model object
    
    Retour: 
    les  etapes du pipeline. 
    """
    
    num_columns = list(X.select_dtypes(exclude=['object']).columns.values.tolist())    
    #categ_columns = list(X.select_dtypes(include=['object']).columns.values.tolist())
    num_pipeline = SimpleImputer(strategy='constant')
    #categ_pipeline = OneHotEncoder(handle_unknown='ignore')
    #MinMaxScaler()
    

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', num_pipeline, num_columns),
            ], remainder='passthrough'
    )
# RFE(estimator=DecisionTreeClassifier(), n_features_to_select=9)RFECV(estimator=XGBClassifier())RandomForestClassifier()
#SMOTE(random_state=1, k_neighbors=4)BorderlineSMOTE(random_state=0)
    steps_pipeline = imbpipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote',SMOTE(random_state=1) ),
        ('scaler', MinMaxScaler()),
        #('feature_selection', RFE(estimator=DecisionTreeClassifier(), n_features_to_select=9)),
        
        ('model', model)
    ])
    
    return steps_pipeline

stacked_models = []
stacked_models.append(('XGBClassifier',XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic')))
stacked_models.append(('LGBMClassifier', LGBMClassifier()))
stacked_models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
stacked_model = VotingClassifier(stacked_models)
sortie_pipeline=features_names = go_inside_pipeline_for_app(X_train, stacked_model)
sortie_pipeline.fit(X_train, y_train)
with open('model_app.pkl', 'wb') as f:
    pickle.dump(sortie_pipeline, f)
y_pred = sortie_pipeline.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
#f1_score = f1_score(y_test, y_pred)
print('ROC/AUC:', roc_auc)
print('Accuracy:', accuracy)
#print('F1 score:', f1_score)

ROC/AUC: 0.9056506849315068
Accuracy: 0.9635294117647059


Hyperparamet

Prends du temps

In [ ]:
params = {'XGBClassifier__n_estimators' : [25, 50,75,100,130,150],'XGBClassifier__learning_rate': [0.005 ,0.05, 0.5, 1.5],'XGBClassifier__max_depth': [2, 4, 6, 8],
'XGBClassifier__max_features': [10, 12, 17] ,
'CatBoostClassifier__depth': [4,8],'CatBoostClassifier__learning_rate' : [0.01,0.02],'CatBoostClassifier__iterations': [10, 20,30,40]   }     
grid = GridSearchCV(estimator=stacked_model, param_grid=params, cv=5)

grid.fit(X_train,y_train)